# ROS and Unit Test

The ROS unittest example was from the Duckietown materials.

## Installation and Setup

### Install ROS 

We will use ROS melodic with Python 2

In [1]:
%%bash
sh -c 'echo "deb http://packages.ros.org/ros/ubuntu $(lsb_release -sc) main" > /etc/apt/sources.list.d/ros-latest.list'
apt-key adv --keyserver 'hkp://keyserver.ubuntu.com:80' --recv-key C1CF6E31E6BADE8868B172B4F42ED6FBAB17C654
apt update  &> /dev/null
apt install ros-melodic-ros-base &> /dev/null

sudo rosdep init  &> /dev/null
rosdep update  &> /dev/null

apt install python-rosinstall  &> /dev/null
apt install python-catkin-tools  &> /dev/null
apt install ros-melodic-ros-numpy ros-melodic-vision-opencv &> /dev/null

Process is terminated.


In [2]:
!sudo apt-get install libeigen3-dev ros-melodic-theora-image-transport ros-melodic-image-publisher

E: Could not get lock /var/lib/dpkg/lock-frontend - open (11: Resource temporarily unavailable)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), is another process using it?


### Setup Python Path

So that we could use rospy, rosbag etc in the notebook.


In [3]:
import sys
import os

sys.path.append('/opt/ros/melodic/lib/python2.7/dist-packages/')
print(sys.path)

os.environ['PATH'] += ':/opt/ros/melodic/bin'
print(os.environ['PATH'])

['', '/env/python', '/usr/lib/python2.7', '/usr/lib/python2.7/plat-x86_64-linux-gnu', '/usr/lib/python2.7/lib-tk', '/usr/lib/python2.7/lib-old', '/usr/lib/python2.7/lib-dynload', '/usr/local/lib/python2.7/dist-packages', '/usr/lib/python2.7/dist-packages', '/usr/local/lib/python2.7/dist-packages/IPython/extensions', '/root/.ipython', '/opt/ros/melodic/lib/python2.7/dist-packages/']
/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/opt/bin:/opt/ros/melodic/bin


### Create a workspace

In [4]:
!mkdir -p /content/catkin_ws/src

## Get Ready the AprilTags Repos



### apriltags_ros 

We will use the ROS package, widely used in the robotics community. There are two ROS packages:
* apriltags: the C++ code as library
* apriltags_ros: also C++ with ROS publisher and subscriber.

General Info about April Tags https://april.eecs.umich.edu/wiki/AprilTags

In [5]:
%cd /content/catkin_ws/src
!git clone https://github.com/Sensing-Intelligent-System/apriltags_ros.git

/content/catkin_ws/src
Cloning into 'apriltags_ros'...
remote: Enumerating objects: 440, done.
remote: Total 440 (delta 0), reused 0 (delta 0), pack-reused 440
Receiving objects: 100% (440/440), 597.51 KiB | 13.28 MiB/s, done.
Resolving deltas: 100% (233/233), done.


### apriltags_ros_test

We will use another repo, originally located in the Duckietown repo.

In [6]:
%cd /content/catkin_ws/src
!git clone https://github.com/Sensing-Intelligent-System/apriltags_ros_test.git

/content/catkin_ws/src
Cloning into 'apriltags_ros_test'...
remote: Enumerating objects: 157, done.
remote: Counting objects: 100% (157/157), done.
remote: Compressing objects: 100% (117/117), done.
remote: Total 157 (delta 79), reused 98 (delta 33), pack-reused 0
Receiving objects: 100% (157/157), 544.28 KiB | 30.24 MiB/s, done.
Resolving deltas: 100% (79/79), done.


### catkin_make

Now we have two ROS packages under src/. Let's compile with catkin_make

In [7]:
%cd /content/catkin_ws
!source /opt/ros/melodic/setup.bash && catkin_make

/content/catkin_ws
/bin/bash: /opt/ros/melodic/setup.bash: No such file or directory


## ROS Test

### The tests folder

In the tests filder, again you could see a pair of tester_node (.py and .test). You could also find two images files, which will be loaded during testing.

In [8]:
!ls /content/catkin_ws/src/apriltags_ros_test/tests

20160508-apriltag108-starducks.png  apriltags_tester_node.py
20210315-apriltag163-stop.jpg	    apriltags_tester_node.test


### apriltags_tester_node.py

In [9]:
!cat /content/catkin_ws/src/apriltags_ros_test/tests/apriltags_tester_node.py

#!/usr/bin/env python
import rospy
import unittest
import rostest
from apriltags_ros.msg import AprilTagDetectionArray
from sensor_msgs.msg import CameraInfo, Image
from cv_bridge import CvBridge
import cv2
from tf import transformations as tr
import numpy as np

class ApriltagsTesterNode(unittest.TestCase):
    def setup(self):
        # Setup the node
        rospy.init_node('apriltags_tester_node', anonymous=False)
        self.msg_received = False
        self.msg_tags = AprilTagDetectionArray()

        # Setup the publisher and subscriber
        self.pub_raw  = rospy.Publisher("~image_rect", Image, queue_size=1, latch=True)
        self.pub_info  = rospy.Publisher("~camera_info", CameraInfo, queue_size=1, latch=True)
        self.sub_tag = rospy.Subscriber( "~apriltags", AprilTagDetectionArray, self.tagCallback)

        # Wait for the node  to finish starting up
        timeout = rospy.Time.now() + rospy.Duration(5) # Wait at most 5 seconds for the node to come up
        while

### apriltags_tester_node.test

In [10]:
!cat /content/catkin_ws/src/apriltags_ros_test/tests/apriltags_tester_node.test

<launch>
    <arg name="pkg_name" value="apriltags_ros"/>
    <arg name="node_name" default="apriltags_tester_node"/>

    <arg name="veh" default="testbot" doc="Name of vehicle. ex: megaman"/>
    <!--<arg name="filename" default="$(find apriltags_ros_test)/tests/20160408-226-intersection_apriltags_24_60-amadobot.png" />-->
    <arg name="filename" default="$(find apriltags_ros_test)/tests/20160508-apriltag108-starducks.png" />
    <!--<arg name="filename" default="$(find apriltags_ros_test)/tests/20210315-apriltag163-stop.jpg" />-->

    <!-- Run the node -->
    <include file="$(find apriltags_ros_test)/launch/apriltag_detector_node.launch">
        <arg name="veh" value="$(arg veh)"/>
    </include>

    <!-- Run unit test -->
    <remap from="apriltags_tester_node/image_rect" to="image_rect" />
    <remap from="apriltags_tester_node/camera_info" to="camera_info" />
    <remap from="apriltags_tester_node/apriltags" to="tag_detections" />
    <test test-name="apriltags_tester_node" 

### Run rostest

Run rostest for apriltags_ros_test apriltags_tester_node.test

In [11]:
!source /content/catkin_ws/devel/setup.bash && rostest apriltags_ros_test apriltags_tester_node.test

/bin/bash: /content/catkin_ws/devel/setup.bash: No such file or directory
